# Homework 2: training pipeline

This code will test your homework 2 solutions by using them in a complete ML pipeline. You should run this code in order to tune your model and save your model weights (which will also be uploaded as part of your solution)

In [ ]:
# Download the training data from the homework2 folder:
# unzip using tar xzvvf nsynth_subset.tar.gz
# (this is a small subset of the "nsynth" dataset: https://magenta.tensorflow.org/datasets/nsynth)

In [1]:
import homework2

c:\ProgramData\anaconda3\Lib\site-packages\paramiko\transport.py:219: CryptographyDeprecationWarning: Blowfish has been deprecated and will be removed in a future release
  "class": algorithms.Blowfish,


### Install and Load Required Libraries  

In [ ]:
# !pip install librosa
# !pip install torch
# !pip install glob
# !pip install numpy

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as nnF
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import numpy as np
import librosa
import random
import glob

In [3]:
BATCH_SIZE = 16
torch.use_deterministic_algorithms(True)

In [4]:
if not len(homework2.audio_paths):
    print("You probably need to set the dataroot folder correctly")

In [ ]:
# Some helper functions. These are the same as what the autograder runs.

In [5]:
# Split dataset into train / valid / test
def split_data(waveforms, labels, train_ratio=0.7, valid_ratio=0.15):
    assert(train_ratio + valid_ratio < 1)
    test_ratio = 1 - (train_ratio + valid_ratio)
    N = len(waveforms)
    Ntrain = int(N * train_ratio)
    Nvalid = int(N * valid_ratio)
    Ntest = int(N * test_ratio)
    Wtrain = waveforms[:Ntrain]
    Wvalid = waveforms[Ntrain:Ntrain + Nvalid]
    Wtest = waveforms[Ntrain + Nvalid:]
    ytrain = labels[:Ntrain]
    yvalid = labels[Ntrain:Ntrain + Nvalid]
    ytest = labels[Ntrain + Nvalid:]
    return Wtrain,Wvalid,Wtest,ytrain,yvalid,ytest

In [6]:
def process_data(W, feature_function):
    return [feature_function(path) for path in W]

In [7]:
class InstrumentDataset(Dataset):
    def __init__(self, features, labels):
        self.features = features
        self.labels = labels
        
    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        features = self.features[idx]
        label = self.labels[idx]

        return features, torch.tensor(label, dtype=torch.long)

In [8]:
class Loaders():
    def __init__(self, waveforms, labels, feature_function, seed = 0):
        torch.manual_seed(seed)
        random.seed(seed)
        self.Wtrain, self.Wvalid, self.Wtest, self.ytrain, self.yvalid, self.ytest = split_data(waveforms, labels)
        
        self.Xtrain = process_data(self.Wtrain, feature_function)
        self.Xvalid = process_data(self.Wvalid, feature_function)
        self.Xtest = process_data(self.Wtest, feature_function)
        
        self.dataTrain = InstrumentDataset(self.Xtrain, self.ytrain)
        self.dataValid = InstrumentDataset(self.Xvalid, self.yvalid)
        self.dataTest = InstrumentDataset(self.Xtest, self.ytest)
        
        self.loaderTrain = DataLoader(self.dataTrain, batch_size=BATCH_SIZE, shuffle=False, num_workers=0)
        self.loaderValid = DataLoader(self.dataValid, batch_size=BATCH_SIZE, shuffle=False, num_workers=0)
        self.loaderTest = DataLoader(self.dataTest, batch_size=BATCH_SIZE, shuffle=False, num_workers=0)

In [9]:
class Pipeline():
    def __init__(self, module, learning_rate, seed = 0):
        # These two lines will (mostly) make things deterministic.
        # You're welcome to modify them to try to get a better solution.
        torch.manual_seed(seed)
        random.seed(seed)

        self.device = torch.device("cpu") # Can change this if you have a GPU, but the autograder will use CPU
        self.criterion = nn.CrossEntropyLoss()
        
        self.model = module.to(self.device)
        self.optimizer = optim.Adam(self.model.parameters(), lr=learning_rate)

    def evaluate(self, loader, which = "valid"):
        self.model.eval()

        correct = 0
        total = 0

        with torch.no_grad():
            for inputs, labels in loader:
                inputs, labels = inputs.to(self.device), labels.to(self.device)

                outputs = self.model(inputs)
                #loss = criterion(outputs, labels) # validation loss

                _, predicted = torch.max(outputs.data, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()

        acc = correct / total
        
        return acc
    
    def train(self, loaders,
          num_epochs=1, # Train for a single epoch by default
          model_path=None): # (Optionally) provide a path to save the best model
        val_acc = 0
        best_val_acc = 0
        for epoch in range(num_epochs):
            self.model.train()
            
            losses = []

            for inputs, labels in loaders.loaderTrain:
                inputs, labels = inputs.to(self.device), labels.to(self.device)

                self.optimizer.zero_grad()
                outputs = self.model(inputs)
                loss = self.criterion(outputs, labels)
                loss.backward()
                self.optimizer.step()
                losses.append(float(loss))
            
            self.model.eval()
            val_acc = self.evaluate(loaders.loaderValid)
            print("Epoch " + str(epoch) + ", loss = " + str(sum(losses)/len(losses)) +\
                  ", validation accuracy = " + str(val_acc))

            if val_acc > best_val_acc:
                best_val_acc = val_acc
                if (model_path):
                    torch.save(self.model.state_dict(), model_path)
        print("Final validation accuracy = " + str(val_acc) + ", best = " + str(best_val_acc))
        return val_acc, best_val_acc

    def load(self, path):
        self.model.load_state_dict(torch.load(path, weights_only=True))

In [ ]:
# The function below is the basis of how the autograder tests your code. Try to understand this one.

In [10]:
def test(waveforms, labels, feature_func, classifier, learning_rate, path):
    print("Extracting features...")
    test_loaders = Loaders(waveforms, labels, feature_func)
    test_pipeline = Pipeline(classifier, learning_rate)
    
    # Note: the autograder will not run this line: it will just load your saved model (next line)
    acc, best_acc = test_pipeline.train(test_loaders, 10, path)
    
    test_pipeline.load(path)
    test_acc = test_pipeline.evaluate(test_loaders.loaderTest)
    print("Test accuracy = " + str(test_acc))

In [ ]:
# 1. Paths, labels, waveforms

In [ ]:
# Once you've written the corresponding code in homework2.py, print these out or visualize them if you want
homework2.waveforms
homework2.labels

In [ ]:
# 2. MFCC

In [ ]:
test(homework2.waveforms,
     homework2.labels,
     homework2.extract_mfcc,
     homework2.MLPClassifier(),
     0.0001,
     "best_mlp_model.weights")

In [ ]:
# 3. Spectrogram

In [11]:
test(homework2.waveforms,
     homework2.labels,
     homework2.extract_spec,
     homework2.SimpleCNN(),
     0.0001,
     "best_spec_model.weights")

Extracting features...
Epoch 0, loss = 0.6366069515546163, validation accuracy = 0.959349593495935
Epoch 1, loss = 0.5683458083205752, validation accuracy = 0.9186991869918699
Epoch 2, loss = 0.543917141854763, validation accuracy = 0.9186991869918699
Epoch 3, loss = 0.5245005157258775, validation accuracy = 0.9186991869918699
Epoch 4, loss = 0.5037712587250603, validation accuracy = 0.9186991869918699
Epoch 5, loss = 0.4797395376695527, validation accuracy = 0.9186991869918699
Epoch 6, loss = 0.45111973997619415, validation accuracy = 0.9512195121951219
Epoch 7, loss = 0.4203209521041976, validation accuracy = 0.9512195121951219
Epoch 8, loss = 0.3906990844342444, validation accuracy = 0.9512195121951219
Epoch 9, loss = 0.363347615632746, validation accuracy = 0.9512195121951219
Final validation accuracy = 0.9512195121951219, best = 0.959349593495935
Test accuracy = 0.9354838709677419


In [ ]:
# 4. Mel-spectrogram

In [ ]:
test(homework2.waveforms,
     homework2.labels,
     homework2.extract_mel,
     homework2.SimpleCNN(),
     0.0001,
     "best_mel_model.weights")

In [ ]:
# 5. Constant-Q

In [15]:
test(homework2.waveforms,
     homework2.labels,
     homework2.extract_q,
     homework2.SimpleCNN(),
     0.0001,
     "best_q_model.weights")

Extracting features...
Epoch 0, loss = 0.619789547390408, validation accuracy = 0.7398373983739838
Epoch 1, loss = 0.49342630472448135, validation accuracy = 0.7967479674796748
Epoch 2, loss = 0.42306200828817153, validation accuracy = 0.8211382113821138
Epoch 3, loss = 0.3681887197825644, validation accuracy = 0.8455284552845529
Epoch 4, loss = 0.3273221444752481, validation accuracy = 0.8699186991869918
Epoch 5, loss = 0.2922023754152987, validation accuracy = 0.8699186991869918
Epoch 6, loss = 0.26237168204453254, validation accuracy = 0.8699186991869918
Epoch 7, loss = 0.2374090879327721, validation accuracy = 0.8943089430894309
Epoch 8, loss = 0.21657439486847985, validation accuracy = 0.9186991869918699
Epoch 9, loss = 0.19876059020558992, validation accuracy = 0.926829268292683
Epoch 10, loss = 0.1834026457120975, validation accuracy = 0.9349593495934959
Epoch 11, loss = 0.1698511474662357, validation accuracy = 0.9512195121951219
Epoch 12, loss = 0.15763221929470697, validation

In [ ]:
# 6. Pitch shift

In [17]:
test(homework2.augmented_waveforms,
     homework2.augmented_labels,
     homework2.extract_q,
     homework2.SimpleCNN(),
     0.0001,
     "best_augmented_model.weights")

Extracting features...
Epoch 0, loss = 0.5107849968804253, validation accuracy = 0.8319783197831978
Epoch 1, loss = 0.3665180015895102, validation accuracy = 0.9105691056910569
Epoch 2, loss = 0.29963376511026313, validation accuracy = 0.9512195121951219
Epoch 3, loss = 0.25356483473270025, validation accuracy = 0.986449864498645
Epoch 4, loss = 0.22008519874954666, validation accuracy = 0.991869918699187
Epoch 5, loss = 0.1942694338935393, validation accuracy = 0.994579945799458
Epoch 6, loss = 0.1731550715242823, validation accuracy = 0.994579945799458
Epoch 7, loss = 0.15451376526444047, validation accuracy = 0.997289972899729
Epoch 8, loss = 0.13812733772728178, validation accuracy = 0.994579945799458
Epoch 9, loss = 0.12375499304659941, validation accuracy = 0.997289972899729
Final validation accuracy = 0.997289972899729, best = 0.997289972899729
Test accuracy = 0.9972972972972973


In [ ]:
# 7. Extend your model to handle four classes and creatively improve its performance

In [ ]:
test(homework2.waveforms,
     homework2.labels_7,
     homework2.feature_func_7,
     homework2.model_7,
     0.0001,
     "best_model_7.weights")